Because the datasets are SO large (especially the Multiome dataset), instead of running both parts of the project in one notebook (and risk Kaggle running out of storage space then resetting all progress), it is more convenient to separate the multiome and citeseq parts of the project, then later merge the predicted outputs from the two parts together.

This notebook concerns itself with the multiome portion.

# First, all the basic imports and file names which may or may not be used is loaded in essentially as a header

In [1]:
!pip install tables

In [2]:
import os, gc, pickle, datetime, scipy.sparse
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from colorama import Fore, Back, Style

from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD,PCA
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from cycler import cycler
from IPython.display import display

import scipy.sparse

In [3]:
# Directory of the data
DATA_DIR = "/kaggle/input/open-problems-multimodal/"
FP_CELL_METADATA = os.path.join(DATA_DIR,"metadata.csv")

FP_CITE_TRAIN_INPUTS = os.path.join(DATA_DIR,"train_cite_inputs.h5")
FP_CITE_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_cite_targets.h5")
FP_CITE_TEST_INPUTS = os.path.join(DATA_DIR,"test_cite_inputs.h5")

FP_MULT_TRAIN_INPUTS = os.path.join(DATA_DIR,"train_multi_inputs.h5")
FP_MULT_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_multi_targets.h5")
FP_MULT_TEST_INPUTS = os.path.join(DATA_DIR,"test_multi_inputs.h5")

FP_MULT_TRAIN_TARGETS_idx = "../input/multimodal-single-cell-as-sparse-matrix/train_multi_targets_idxcol.npz"
FP_MULT_TRAIN_TARGETS_sparse = "../input/multimodal-single-cell-as-sparse-matrix/train_multi_targets_values.sparse.npz"
FP_MULT_TRAIN_INPUTS_idx = "../input/multimodal-single-cell-as-sparse-matrix/train_multi_inputs_idxcol.npz"
FP_MULT_TRAIN_INPUTS_sparse = "../input/multimodal-single-cell-as-sparse-matrix/train_multi_inputs_values.sparse.npz"
FP_MULT_TEST_INPUTS_idx = "../input/multimodal-single-cell-as-sparse-matrix/test_multi_inputs_idxcol.npz"
FP_MULT_TEST_INPUTS_sparse = "../input/multimodal-single-cell-as-sparse-matrix/test_multi_inputs_values.sparse.npz"

FP_SUBMISSION = os.path.join(DATA_DIR,"sample_submission.csv")
FP_EVALUATION_IDS = os.path.join(DATA_DIR,"evaluation_ids.csv")

FP_EVALUATION_IDS_parquet = "../input/multimodal-single-cell-as-sparse-matrix/evaluation.parquet"


# Multiome

Now the multiome portion begins

Model from: https://www.kaggle.com/code/xiafire/msci-multiome-5-steps-x-5-folds-25-models

## Load in the data

In [4]:
%%time
# 1 min 54 s

train_inputs = scipy.sparse.load_npz(FP_MULT_TRAIN_INPUTS_sparse)
train_inputs = train_inputs.astype('float16', copy=False)

train_targets = scipy.sparse.load_npz(FP_MULT_TRAIN_TARGETS_sparse)

test_inputs = scipy.sparse.load_npz(FP_MULT_TEST_INPUTS_sparse)


train_target_cols = np.load(FP_MULT_TRAIN_TARGETS_idx,
                    allow_pickle=True)["columns"]

test_input_rows = np.load(FP_MULT_TEST_INPUTS_idx,
                  allow_pickle=True)["index"]

CPU times: user 1min 15s, sys: 7.84 s, total: 1min 23s
Wall time: 2min 6s


## Perform SVD

In [5]:
num_components = 256 # this will be the setting for n_components
random_state_num = 4060 # random state number

In [6]:
%%time
# 34 minutes 24 seconds

# reduce the dimensionality of the training inputs
pca_inputs = TruncatedSVD(n_components=num_components, random_state=random_state_num)
train_inputs = pca_inputs.fit_transform(train_inputs)

print(pca_inputs.explained_variance_ratio_.sum())

0.014594905
CPU times: user 45min 50s, sys: 34.9 s, total: 46min 25s
Wall time: 45min 45s


In [7]:
%%time
# 5 minutes 58 seconds

# Reduce the dimensionality of the training targets
pca_targets = TruncatedSVD(n_components=num_components, random_state=random_state_num)
train_target = pca_targets.fit_transform(train_targets)
print(pca_targets.explained_variance_ratio_.sum())

0.13660844
CPU times: user 13min 35s, sys: 10.2 s, total: 13min 45s
Wall time: 13min 30s


## Multiome learning model

From: https://www.kaggle.com/code/xiafire/msci-multiome-5-steps-x-5-folds-25-models

In [8]:
# Use kernel ridge regression
from sklearn.gaussian_process.kernels import RBF
from sklearn.kernel_ridge import KernelRidge
kernel = RBF(length_scale = 10)
krr = KernelRidge(alpha=0.2, kernel=kernel)

In [9]:
n = 5 # 5 folds

correlation score from MSCI CITEseq Keras Quickstart by AMBROSM

In [10]:
def correlation_score(y_true, y_pred):
    """Scores the predictions according to the competition rules. 
    
    It is assumed that the predictions are not constant.
    
    Returns the average of each sample's Pearson correlation coefficient"""
    if type(y_true) == pd.DataFrame: y_true = y_true.values
    if type(y_pred) == pd.DataFrame: y_pred = y_pred.values
    if y_true.shape != y_pred.shape: raise ValueError("Shapes are different.")
    corrsum = 0
    for i in range(len(y_true)):
        corrsum += np.corrcoef(y_true[i], y_pred[i])[1, 0]
    return corrsum / len(y_true)

In [11]:
%%time
# 47 min 59 s

# Perform training
from sklearn.model_selection import KFold
import os, gc, pickle

np.random.seed(random_state_num)
all_row_indices = np.arange(train_inputs.shape[0])
np.random.shuffle(all_row_indices)

kf = KFold(n_splits=5, shuffle=True, random_state=random_state_num)

index = 0
score = []

d = train_inputs.shape[0]//n
for i in range(0, n*d, d):
    print(f'start [{i}:{i+d}]')
    ind = all_row_indices[i:i+d]    
    for idx_tr, idx_va in kf.split(ind):
        X = train_inputs[ind]
        Y = train_target[ind] #.todense()
        Yva = train_targets[ind][idx_va]
        Xtr, Xva = X[idx_tr], X[idx_va]
        Ytr = Y[idx_tr]
        del X, Y
        gc.collect()
        
        print('Train...')
        model = krr #Ridge(copy_X=False)
        model.fit(Xtr, Ytr)
        del Xtr, Ytr
        gc.collect()
        
        s = correlation_score(Yva.todense(), model.predict(Xva)@pca_targets.components_)
        score.append(s)
        print(index, s)
        del Xva, Yva
        gc.collect()
        
        pkl_filename = f"model{index:02d}.pkl"
        index += 1
        with open(pkl_filename, 'wb') as file:
            pickle.dump(model, file)
    gc.collect()

start [0:21188]
Train...
0 0.6666178015950062
Train...
1 0.6667195887734685
Train...
2 0.6665781412397265
Train...
3 0.6656830396416116
Train...
4 0.6664251980752822
start [21188:42376]
Train...
5 0.6659853108094034
Train...
6 0.6652651114289787
Train...
7 0.6673117190701843
Train...
8 0.6660474264420619
Train...
9 0.6663596736482712
start [42376:63564]
Train...
10 0.6662188179042828
Train...
11 0.6656253493463776
Train...
12 0.6651468179658976
Train...
13 0.6662368463272412
Train...
14 0.6661533059737205
start [63564:84752]
Train...
15 0.6661340197737444
Train...
16 0.6666357666504569
Train...
17 0.6643476469057601
Train...
18 0.6660095820698346
Train...
19 0.6669874878528076
start [84752:105940]
Train...
20 0.6667461455250722
Train...
21 0.6666717786747607
Train...
22 0.6658150959604827
Train...
23 0.6674076780976589
Train...
24 0.6688406420292216
CPU times: user 1h 35min 20s, sys: 4min 3s, total: 1h 39min 24s
Wall time: 59min 56s


In [12]:
# Delete variables to free up space
del train_inputs # Multiome training data, has SVD applied (inputs)
del train_targets # Multiome training data, no SVD applied (targets)
del train_target # Multiome training data after SVD (targets)
gc.collect()

21

## Predictions for multiome

In [13]:
%%time
# 1 min 12 s

# For test inputs, match the pca transform of the training inputs
test_inputs = pca_inputs.transform(test_inputs)

CPU times: user 1min 12s, sys: 110 ms, total: 1min 12s
Wall time: 1min 12s


In [14]:
%%time
# less than a second

test_len = test_inputs.shape[0]
d = test_len//n
x = []
for i in range(n):
    x.append(test_inputs[i*d:i*d+d])
    
del test_inputs
gc.collect()

CPU times: user 125 ms, sys: 975 µs, total: 126 ms
Wall time: 126 ms


42

In [15]:
%%time
# 1 h 34 min 47 s

preds = np.zeros((test_len, 23418), dtype='float16')
for i,xx in enumerate(x):
    for ind in range(index): # index gives the number of models (25)
        print(ind, end=' ')
        with open(f'model{ind:02}.pkl', 'rb') as file:
            model = pickle.load(file)
        preds[i*d:i*d+d,:] += (model.predict(xx)@pca_targets.components_)/index
        gc.collect()
    print('')
    del xx
gc.collect()

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 
CPU times: user 2h 34min 58s, sys: 6min 15s, total: 2h 41min 14s
Wall time: 2h 14min 56s


0

In [16]:
del x
gc.collect()

21

In [17]:
%%time
# 4.38 s
np.save('preds.npy', preds) # save predictions

CPU times: user 2.59 ms, sys: 3.67 s, total: 3.67 s
Wall time: 3.67 s


In [18]:
%%time
# less than a minute
preds = preds.astype('float16', copy=False)
preds

CPU times: user 119 µs, sys: 0 ns, total: 119 µs
Wall time: 123 µs


array([[1.192 , 0.574 , 0.461 , ..., 1.675 , 2.262 , 4.277 ],
       [1.344 , 0.6846, 0.8486, ..., 2.518 , 2.89  , 4.242 ],
       [1.178 , 0.4863, 0.9497, ..., 1.683 , 2.213 , 3.693 ],
       ...,
       [0.401 , 0.4568, 0.703 , ..., 1.809 , 0.9077, 3.248 ],
       [0.4253, 0.5137, 0.541 , ..., 1.927 , 1.451 , 3.684 ],
       [1.214 , 0.911 , 0.3657, ..., 2.572 , 8.305 , 6.375 ]],
      dtype=float16)

## Submission format and save

In [19]:
%%time
# 42 seconds

# Read the table of rows and columns required for submission
eval_ids = pd.read_parquet(FP_EVALUATION_IDS_parquet)
# Convert the string columns to more efficient categorical types
#eval_ids.cell_id = eval_ids.cell_id.apply(lambda s: int(s, base=16))
eval_ids.cell_id = eval_ids.cell_id.astype(pd.CategoricalDtype())
eval_ids.gene_id = eval_ids.gene_id.astype(pd.CategoricalDtype())

CPU times: user 33.7 s, sys: 18.3 s, total: 52 s
Wall time: 47.6 s


In [20]:
# Prepare an empty series which will be filled with predictions
submission = pd.Series(name='target',
                       index=pd.MultiIndex.from_frame(eval_ids), 
                       dtype=np.float32)
submission

row_id    cell_id       gene_id        
0         c2150f55becb  CD86              NaN
1         c2150f55becb  CD274             NaN
2         c2150f55becb  CD270             NaN
3         c2150f55becb  CD155             NaN
4         c2150f55becb  CD112             NaN
                                           ..
65744175  2c53aa67933d  ENSG00000134419   NaN
65744176  2c53aa67933d  ENSG00000186862   NaN
65744177  2c53aa67933d  ENSG00000170959   NaN
65744178  2c53aa67933d  ENSG00000107874   NaN
65744179  2c53aa67933d  ENSG00000166012   NaN
Name: target, Length: 65744180, dtype: float32

In [21]:
%%time
# 34.5 ms

cell_dict = dict((k,v) for v,k in enumerate(test_input_rows)) 
assert len(cell_dict)  == len(test_input_rows)

gene_dict = dict((k,v) for v,k in enumerate(train_target_cols))
assert len(gene_dict) == len(train_target_cols)

CPU times: user 52.8 ms, sys: 0 ns, total: 52.8 ms
Wall time: 53 ms


In [22]:
%%time
# 2.92 s

eval_ids_cell_num = eval_ids.cell_id.apply(lambda x:cell_dict.get(x, -1))
eval_ids_gene_num = eval_ids.gene_id.apply(lambda x:gene_dict.get(x, -1))

valid_multi_rows = (eval_ids_gene_num !=-1) & (eval_ids_cell_num!=-1)

CPU times: user 1.71 s, sys: 985 ms, total: 2.7 s
Wall time: 2.42 s


In [23]:
%%time
# 4.78 s

submission.iloc[valid_multi_rows] = preds[eval_ids_cell_num[valid_multi_rows].to_numpy(),
eval_ids_gene_num[valid_multi_rows].to_numpy()]

CPU times: user 1.73 s, sys: 1.62 s, total: 3.35 s
Wall time: 3.35 s


In [24]:
del eval_ids_cell_num, eval_ids_gene_num, valid_multi_rows, eval_ids, test_input_rows, train_target_cols
gc.collect()

55

In [25]:
submission

row_id    cell_id       gene_id        
0         c2150f55becb  CD86                     NaN
1         c2150f55becb  CD274                    NaN
2         c2150f55becb  CD270                    NaN
3         c2150f55becb  CD155                    NaN
4         c2150f55becb  CD112                    NaN
                                             ...    
65744175  2c53aa67933d  ENSG00000134419    14.890625
65744176  2c53aa67933d  ENSG00000186862     0.086182
65744177  2c53aa67933d  ENSG00000170959     0.116577
65744178  2c53aa67933d  ENSG00000107874     2.857422
65744179  2c53aa67933d  ENSG00000166012    12.351562
Name: target, Length: 65744180, dtype: float32

In [26]:
submission.reset_index()

,row_id,cell_id,gene_id,target
0,0,c2150f55becb,CD86,NaN
1,1,c2150f55becb,CD274,NaN
2,2,c2150f55becb,CD270,NaN
3,3,c2150f55becb,CD155,NaN
4,4,c2150f55becb,CD112,NaN
...,...,...,...,...
65744175,65744175,2c53aa67933d,ENSG00000134419,14.890625
65744176,65744176,2c53aa67933d,ENSG00000186862,0.086182
65744177,65744177,2c53aa67933d,ENSG00000170959,0.116577
65744178,65744178,2c53aa67933d,ENSG00000107874,2.857422


In [27]:
%%time

submission = submission.reset_index().drop(['cell_id', 'gene_id'], axis=1)
submission

CPU times: user 2.36 s, sys: 1.06 s, total: 3.41 s
Wall time: 3.42 s


,row_id,target
0,0,NaN
1,1,NaN
2,2,NaN
3,3,NaN
4,4,NaN
...,...,...
65744175,65744175,14.890625
65744176,65744176,0.086182
65744177,65744177,0.116577
65744178,65744178,2.857422


In [28]:
%%time

submission.set_index('row_id', inplace=True)
submission


CPU times: user 717 µs, sys: 27 µs, total: 744 µs
Wall time: 755 µs


,target
row_id,
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
65744175,14.890625
65744176,0.086182
65744177,0.116577


In [29]:
%%time
# Approximately 2 minutes

# save as csv
submission.to_csv('multiome_only_256.csv')

CPU times: user 2min 2s, sys: 3.69 s, total: 2min 6s
Wall time: 2min 6s
